In [459]:
import pandas as pd
from datetime import date
import numpy as np

In [460]:
def get_label(s):
    s = s.split(':')
    if s[0] == 'null':
        return 0
    elif (date(int(s[0][0:4]), int(s[0][4:6]), int(s[0][6:8])) - date(int(s[1][0:4]), int(s[1][4:6]),
                                                                      int(s[1][6:8]))).days <= 15:
        return 1
    else:
        return -1

In [461]:
def mergeData(coupon_feature_file,merchant_feature_file,user_feature_file,user_merchant_file,other_feature_file,file_name,flag):
    coupon3 = pd.read_csv(coupon_feature_file)
    merchant3 = pd.read_csv(merchant_feature_file)
    dataset3 = pd.merge(coupon3, merchant3, on='merchant_id', how='left')
    dataset3.ix[:5]

    user3 = pd.read_csv(user_feature_file)
    dataset3 = pd.merge(dataset3, user3, on='user_id', how='left')
    dataset3.ix[:5]

    user_merchant3 = pd.read_csv(user_merchant_file)
    user_merchant3.ix[:5]
    dataset3 = pd.merge(dataset3, user_merchant3, on=['user_id', 'merchant_id'], how='left')
    dataset3.ix[:5]

    other_feature3 = pd.read_csv(other_feature_file)
    dataset3 = pd.merge(dataset3, other_feature3, on=['user_id', 'coupon_id', 'date_received'], how='left')
    dataset3.drop_duplicates(inplace=True)

    dataset3.ix[:5]

    dataset3['is_weekend'] = dataset3.day_of_week.apply(lambda x: 1 if x in (6, 7) else 0)
    weekday_dummies = pd.get_dummies(dataset3.day_of_week)
    weekday_dummies.columns = ['weekday' + str(i + 1) for i in range(weekday_dummies.shape[1])]
    dataset3 = pd.concat([dataset3, weekday_dummies], axis=1)
    
    if flag :
        dataset3['label'] = dataset3.date.astype('str') + ':' + dataset3.date_received.astype('str')
        dataset3.label = dataset3.label.apply(get_label)
        dataset3.drop(['merchant_id', 'day_of_week', 'date', 'date_received', 'coupon_id', 'coupon_count'], axis=1,inplace=True)
    else :
        dataset3.drop(['merchant_id', 'day_of_week', 'coupon_count'], axis=1, inplace=True)
    dataset3 = dataset3.replace('null', np.nan)
    dataset3 = dataset3.replace(np.nan, 0)
    dataset3.ix[:5]
    dataset3.to_csv(file_name, index=None)
    return dataset3

In [462]:
dataset3 = mergeData('data/coupon3_feature.csv','data/merchant3_feature.csv','data/user3_feature.csv',
                  'data/user_merchant3.csv','data/other_feature3.csv','data/dataset3.csv',False)
dataset3.ix[:5]

,user_id,coupon_id,discount_rate,distance,date_received,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,...,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,4129537,9983,0.833333,1,20160712,12,12,30,5,1,...,0.0,0.0,0,0,1,0,0,0,0,0
1,6949378,3429,0.833333,0,20160706,6,6,30,5,1,...,0.0,0.0,0,0,0,1,0,0,0,0
2,2166529,6928,0.900000,5,20160727,27,27,200,20,1,...,0.0,0.0,0,0,0,1,0,0,0,0
3,2166529,1808,0.900000,5,20160727,27,27,100,10,1,...,0.0,0.0,0,0,0,1,0,0,0,0
4,6172162,6500,0.966667,2,20160708,8,8,30,1,1,...,0.0,0.0,0,0,0,0,0,1,0,0
5,4005121,9983,0.833333,0,20160706,6,6,30,5,1,...,0.0,0.0,0,0,0,1,0,0,0,0


In [463]:
dataset2 = mergeData('data/coupon2_feature.csv','data/merchant2_feature.csv','data/user2_feature.csv',
                  'data/user_merchant2.csv','data/other_feature2.csv','data/dataset2.csv',True)
dataset2.ix[:5]

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,...,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1439408,0.866667,1,28,-33,150,20,1,657.0,26.0,...,0.0,1,0,0,0,0,0,1,0,0
1,1439408,0.950000,0,13,-17,20,1,1,14.0,3.0,...,0.0,0,1,0,0,0,0,0,0,0
2,1439408,0.950000,0,16,-45,20,1,1,14.0,3.0,...,0.0,0,1,0,0,0,0,0,0,-1
3,2029232,0.833333,0,30,-31,30,5,1,6307.0,643.0,...,0.0,0,1,0,0,0,0,0,0,0
4,2029232,0.950000,0,19,-42,20,1,1,5.0,0.0,...,0.0,0,0,0,0,1,0,0,0,0
5,2747744,0.800000,0,6,-24,50,10,1,7471.0,390.0,...,0.0,0,1,0,0,0,0,0,0,0


In [464]:
dataset1 = mergeData('data/coupon1_feature.csv','data/merchant1_feature.csv','data/user1_feature.csv',
                  'data/user_merchant1.csv','data/other_feature1.csv','data/dataset1.csv',True)
dataset1.ix[:5]

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,...,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1832624,0.900000,0,29,-62,200,20,1,14962.0,2168.0,...,0.0,0,0,0,0,0,1,0,0,0
1,163606,0.850000,10,21,-70,200,30,1,787.0,105.0,...,0.0,0,0,0,0,1,0,0,0,0
2,4061024,0.900000,10,26,-65,200,20,1,14962.0,2168.0,...,0.0,0,0,1,0,0,0,0,0,0
3,106443,0.833333,0,29,-62,30,5,1,6553.0,408.0,...,0.0,0,0,0,0,0,1,0,0,0
4,114747,0.850000,9,26,-65,200,30,1,787.0,105.0,...,0.0,0,0,1,0,0,0,0,0,0
5,114747,0.850000,9,26,-65,200,30,1,787.0,105.0,...,0.0,0,0,1,0,0,0,0,0,0
